# Attendance Exercise 02

In this exercise we will some of the basics of  object-oriented programming (OOP), which offer a way to bundle together data and functions that operate on that data, modifying the internal _state_ of the object.

To test your solutions, check out the tests at the end of the exercise.

In [1]:
%load_ext autoreload
%autoreload 2
import unittest

# following is needed for Polygon drawing
import matplotlib.pyplot as plt
import numpy as np

from vector_2d import Vector2D
from utils import print_python_code

## 1. Simple class `Person`

In their most simple form classes can be used as a data record. Their data is stored in attributes (member variables, denoted with `self.<varname>` where `self` is stands for the instance of the class).

In [2]:
class Person():
    def __init__(self, first_name, last_name, age):
        if not isinstance(first_name, str) or not isinstance(last_name, str) or not isinstance(age, int):
            raise ValueError
        self.first_name = first_name
        self.last_name = last_name
        self.age = age

    def full_name(self):
        """Returns full name (`<first name> <last name>` as string)"""
        return self.first_name + " " + self.last_name
        
    def say_hello(self):
        """Prints `Hello <first name> <last name>`"""
        print("Hello " + self.full_name())


    def older_than(self, other):
        """Return ``True`` if Person is older than other person"""
        return self.age > other.age

    
    def __gt__(self, other):
        return self.older_than(other)

    
    def __str__(self):
        return self.full_name()

In [3]:
person1 = Person("Brian", "Kernighan", 79)
print(person1)

Brian Kernighan


In [4]:
print(person1.first_name, person1.last_name, person1.age)

Brian Kernighan 79


In [5]:
person1.say_hello()
print(person1)

Hello Brian Kernighan
Brian Kernighan


In [6]:
person2 = Person("Margaret", "Hamilton", 84)
print(person2)

Margaret Hamilton


In [7]:
person2.older_than(person1)

True

In [8]:
person2 > person1

True

In [9]:
import io
from contextlib import redirect_stdout
    
class TestPerson(unittest.TestCase):
    def setUp(self):
        self.person1 = Person("Brian", "Kernighan", 79)
        self.person2 = Person("Margaret", "Hamilton", 84)
    
    def test_attributes(self):
        self.assertEqual(self.person1.first_name, "Brian")
        self.assertEqual(self.person1.last_name, "Kernighan")
        self.assertEqual(self.person1.age, 79)
    
    def test_full_name(self):
        self.assertEqual(self.person2.full_name(), "Margaret Hamilton")
        
    def test_str(self):
        self.assertEqual(str(self.person2), self.person2.full_name())
        
    def test_say_hello(self):
        out = io.StringIO()
        with redirect_stdout(out):
            self.person2.say_hello()
        self.assertEqual(out.getvalue().strip(), "Hello Margaret Hamilton")
    
    def test_older_than(self):
        self.assertFalse(self.person1.older_than(person2))
        self.assertTrue(self.person2.older_than(person1))
        
        
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestPerson)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

test_attributes (__main__.TestPerson) ... ok
test_full_name (__main__.TestPerson) ... ok
test_older_than (__main__.TestPerson) ... ok
test_say_hello (__main__.TestPerson) ... ok
test_str (__main__.TestPerson) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.008s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

## 2. Extending a class

In the lecture you have seen the class `Vector2D`, which I have included in the module `vector_2d`.

In [10]:
from vector_2d import Vector2D
print_python_code(Vector2D)

class Vector2D:
    # class initialisation
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def copy(self):
        return Vector2D(self.x, self.y)

    def __str__(self):
        return (
            "x:" + str(self.x) + ", y:" + str(self.y)
        )  # Human-readable string representation of the vector.

    def __repr__(self):
        return f"{self.__class__.__name__}({self.__str__()})"

    def __add__(self, other):
        return Vector2D(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector2D(self.x - other.x, self.y - other.y)

    def __eq__(self, other):
        return (self.x == other.x) and (self.y == other.y)

    def __mul__(self, other):  # dot product or multiplication by scalar
        if isinstance(other, Vector2D):
            return self.x * other.x + self.y * other.y
        if isinstance(other, (float, int)):
            return Vector2D(self.x * other, self.y * other)
        raise NotImplementedError(
  

That implementation already contains vector addition, substraction and multiplication (both dot product and scalar multiplication). 

In [11]:
v1 = Vector2D(2, 0)
v2 = Vector2D(-2, 2)
print(v1 * 2)
print(v1 + v2)
print(v1 * v2)
print((v1-v2).to_polar())
# play around on your own to see what's supprted

x:4, y:0
x:0, y:2
-4
(4.47213595499958, -0.4636476090008061)


What's still missing is the devision operator `/` for devisision with a scalar. Without it, the following code for example fails:

In [12]:
v1 = Vector2D(2, 0)
v2 = Vector2D(-2, 2)

mean = (v1 + v2) / 2
mean

Vector2D(x:0.0, y:1.0)

- To fix this, to implement the `__truediv__` method. Open the file and edit it in VSCode, adding a function

``` python
def __truediv__(self, other):
    ...
```

- Use this functional to implement a method `normal`, which returns a vector with the same direction, but normalized to unit length
- Implement the method `angle_to` to get the angle between two vectors.

In [13]:
import math

class TestNewVector2D(unittest.TestCase):
    
    def test_truediv(self):
        v = Vector2D(0.5, -4)
        self.assertAlmostEqual(v / 2, Vector2D(0.25, -2))
    
    def test_normal(self):
        v = Vector2D(1, 1)
        nv = v.normal()
        self.assertAlmostEqual(nv, v / math.sqrt(2))
        # test old vector is unchanged
        self.assertEqual(v, Vector2D(1, 1))
    
    def test_angle_to(self):
        v1 = Vector2D(0.5, 0)
        v2 = Vector2D(10, 10)
        angle = v1.angle_to(v2)
        self.assertAlmostEqual(angle, math.pi/4)
        
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestNewVector2D)
runner.run(suite)

test_angle_to (__main__.TestNewVector2D) ... ok
test_normal (__main__.TestNewVector2D) ... ok
test_truediv (__main__.TestNewVector2D) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

## 3. Simple Inheritance

One method to extent existing code without directly editing it, is to _inherit_ a class from it. The _sub-class_ inherits all methods and attributes of the super-class, so we don't have to re-implement or copy-paste the existing code. The syntax for inheritance is
``` python
class SubClass(SuperClass):
```
The sub-class is by convention a _specification_ of the more general super-class. This means that all methods that are supported by the super-class should be supported by the sub-class, but the sub-class can implement methods that the super-class doesn't have. A subclass always _is_ also an instance of the super-class.

Imagine a particle is described by its velocity vector and its mass, but we are not interested in this position. Then, one can represent the particle as sub-class of `Vector2D`. Mind that the `x` and `y` should represent the particle velocity for the purpose of this problem.

- Implement an `__init__(self, vx, vy, mass)` method that sets the `x`/`y`-components of the super-class (hint: use `super().__init__`)  and sets the mass property
- Impliment `momentum()` and `energy()` methods. The `momentum()` should return an instance of `Vector2D`


In [14]:
class Particle(Vector2D):
    """
    A particle with velocity and mass but no position. x and y referer to its velocity components.
    """
    def __init__(self, vx, vy , mass):
        super().__init__(vx, vy)
        self.mass = mass
    
    def energy(self):
        return self.mass*(self.x**2+self.y**2)/2

    def momentum(self):
        return self*self.mass

In [15]:
p = Particle(1, 1, 10)
print(p.mass, p.energy(), p.momentum())

10 10.0 x:10, y:10


In [16]:
isinstance(p, Vector2D)

True

In [17]:
class TestParticle(unittest.TestCase):
    
    def test_is_Vector2D(self):
        p = Particle(1, 0, 2)
        self.assertIsInstance(p, Vector2D)
        
    def test_init(self):
        p = Particle(1, 0, 2)
        self.assertAlmostEqual(p.mass, 2)
        self.assertAlmostEqual(p.x, 1)
        self.assertAlmostEqual(p.y, 0)

    
    def test_momentum(self):
        p = Particle(1, 0, 2)
        self.assertEqual(p.momentum(), Vector2D(2, 0))
    
    def test_energy(self):
        p = Particle(1, 0, 2)
        self.assertEqual(p.energy(), 1)
        
        
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestParticle)
runner.run(suite)

test_energy (__main__.TestParticle) ... ok
test_init (__main__.TestParticle) ... ok
test_is_Vector2D (__main__.TestParticle) ... ok
test_momentum (__main__.TestParticle) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.006s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

Make sure you get an error if you try to instantiate your rectangle with points for a parallelogram